# Import of all pictures into a Pandas Dataframe

Import of libraries

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import cv2

## Data preparation
First count the number of pictures. Then put the paths for all pictures with the class and patient_id in a dataframe entry.

In [9]:
base_path = "data"
folder = os.listdir(base_path)
total_images = 0

#finding the number of total images
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = os.path.join(base_path, patient_id )
        class_path = os.path.join(patient_path, str(c))
        subfiles = os.listdir(class_path)
        total_images += len(subfiles)

#DATAFRAME
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "class"])
k = 0
for n in range(len(folder)):
    #if folder[n] == 'IDC_regular_ps50_idx5':
    #    continue
    patient_id = folder[n]
    patient_path = os.path.join(base_path, patient_id )
    for c in [0,1]:
        class_path = os.path.join(patient_path, str(c))
        subfiles = os.listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            if image_path[-18:] == '.ipynb_checkpoints':
                continue
            data.iloc[k]["path"] = os.path.join(class_path, image_path)
            data.iloc[k]["class"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1  

In [10]:
print(data.tail)
#print(data.loc[11124,'path'])
#print(data.loc[11124])
#plt.imshow(plt.imread(data.loc[11124,'path']))

<bound method NDFrame.tail of        patient_id                                            path class
0           10253  data\10253\0\10253_idx5_x1001_y1001_class0.png     0
1           10253  data\10253\0\10253_idx5_x1001_y1051_class0.png     0
2           10253  data\10253\0\10253_idx5_x1001_y1101_class0.png     0
3           10253  data\10253\0\10253_idx5_x1001_y1151_class0.png     0
4           10253  data\10253\0\10253_idx5_x1001_y1201_class0.png     0
...           ...                                             ...   ...
277519       9383     data\9383\1\9383_idx5_x2051_y901_class1.png     1
277520       9383     data\9383\1\9383_idx5_x2051_y951_class1.png     1
277521       9383    data\9383\1\9383_idx5_x2101_y1001_class1.png     1
277522       9383     data\9383\1\9383_idx5_x2101_y901_class1.png     1
277523       9383     data\9383\1\9383_idx5_x2101_y951_class1.png     1

[277524 rows x 3 columns]>


In [11]:
#Creating a dataframe for each patient
for i in folder:
    if folder[n] == 'IDC_regular_ps50_idx5':
        continue
    df = data[data['patient_id'] == i]
    exec(f'data{i} = df')

In [12]:
#using train_test_split for splitting the data
train_data, test_data = train_test_split(data, test_size=0.3, random_state=30)
print('########## Train data ##########')
print(train_data)
print('########## Test data ##########')
print(test_data)

########## Train data ##########
       patient_id                                            path class
167268      15473   data\15473\0\15473_idx5_x1351_y451_class0.png     0
25485       10282    data\10282\0\10282_idx5_x651_y851_class0.png     0
242394       9181    data\9181\0\9181_idx5_x2551_y1101_class0.png     0
139916      13693   data\13693\0\13693_idx5_x2001_y701_class0.png     0
27823       10288   data\10288\0\10288_idx5_x1051_y451_class0.png     0
...           ...                                             ...   ...
3905        10259   data\10259\0\10259_idx5_x2051_y951_class0.png     0
108969      12935  data\12935\1\12935_idx5_x1151_y1751_class1.png     1
222339       9043    data\9043\1\9043_idx5_x2301_y1051_class1.png     1
46220       10305   data\10305\0\10305_idx5_x1951_y801_class0.png     0
48045       10306  data\10306\0\10306_idx5_x2451_y1151_class0.png     0

[194266 rows x 3 columns]
########## Test data ##########
       patient_id                           

In [13]:
print(data.iloc[873])

patient_id                                            10254
path          data\10254\0\10254_idx5_x1851_y651_class0.png
class                                                     0
Name: 873, dtype: object


In [15]:
parameters_df = pd.DataFrame(columns = ['red', 'green', 'blue', 'gray'])
number_of_pix = 50*50

st = time.time()

for i in range(2500):
    #if data.loc[i,'path'][-18:] == '.ipynb_checkpoints':
    #    continue
    img = plt.imread(data.loc[i,'path'])
    imgGray = cv2.imread(data.loc[i,'path'],0)
    imgGray = cv2.cvtColor(imgGray, cv2.COLOR_BGR2RGB)
    
    gray = sum(sum(imgGray[:,:,0]))/number_of_pix
    red = sum(sum(img[:,:,0]))/number_of_pix
    green = sum(sum(img[:,:,1]))/number_of_pix
    blue = sum(sum(img[:,:,2]))/number_of_pix
    parameters_df.loc[data.loc[i,'path']] = [red,green,blue,gray]
    
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

print(parameters_df)

Execution time: 15.019989013671875 seconds
                                                     red     green      blue  \
data\10253\0\10253_idx5_x1001_y1001_class0.png  0.827936  0.582193  0.756613   
data\10253\0\10253_idx5_x1001_y1051_class0.png  0.875114  0.723770  0.835029   
data\10253\0\10253_idx5_x1001_y1101_class0.png  0.916524  0.848179  0.903890   
data\10253\0\10253_idx5_x1001_y1151_class0.png  0.913735  0.863232  0.911594   
data\10253\0\10253_idx5_x1001_y1201_class0.png  0.914912  0.823939  0.884461   
...                                                  ...       ...       ...   
data\10257\0\10257_idx5_x2251_y801_class0.png   0.674976  0.434455  0.609438   
data\10257\0\10257_idx5_x2251_y851_class0.png   0.698496  0.540002  0.684933   
data\10257\0\10257_idx5_x2301_y1601_class0.png  0.749691  0.733233  0.818891   
data\10257\0\10257_idx5_x2301_y1751_class0.png  0.898943  0.846397  0.886857   
data\10257\0\10257_idx5_x2301_y2001_class0.png  0.897586  0.850609  0.885538 

In [16]:
#Export dataframe to JSON file
parameters_df.to_json(r'parameters_df.json', orient='index')

In [17]:
#Import dataframe from JSON file
jsonStr = "parameters_df.json"
imported_df = pd.read_json(jsonStr, orient ='index')
print(imported_df.tail)

<bound method NDFrame.tail of                                                      red     green      blue  \
data\10253\0\10253_idx5_x1001_y1001_class0.png  0.827936  0.582193  0.756613   
data\10253\0\10253_idx5_x1001_y1051_class0.png  0.875114  0.723770  0.835029   
data\10253\0\10253_idx5_x1001_y1101_class0.png  0.916524  0.848179  0.903890   
data\10253\0\10253_idx5_x1001_y1151_class0.png  0.913735  0.863232  0.911594   
data\10253\0\10253_idx5_x1001_y1201_class0.png  0.914912  0.823939  0.884461   
...                                                  ...       ...       ...   
data\10257\0\10257_idx5_x2251_y801_class0.png   0.674976  0.434455  0.609438   
data\10257\0\10257_idx5_x2251_y851_class0.png   0.698496  0.540002  0.684933   
data\10257\0\10257_idx5_x2301_y1601_class0.png  0.749691  0.733233  0.818891   
data\10257\0\10257_idx5_x2301_y1751_class0.png  0.898943  0.846397  0.886857   
data\10257\0\10257_idx5_x2301_y2001_class0.png  0.897586  0.850609  0.885538   

         